In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/universalsentenceencoderlarge4/saved_model.pb
/kaggle/input/universalsentenceencoderlarge4/variables/variables.index
/kaggle/input/universalsentenceencoderlarge4/variables/variables.data-00000-of-00001
/kaggle/input/google-quest-challenge/test.csv
/kaggle/input/google-quest-challenge/train.csv
/kaggle/input/google-quest-challenge/sample_submission.csv
/kaggle/input/robertaqa/test/roberta-base-features-question-5-7.h5py.data-00001-of-00002
/kaggle/input/robertaqa/test/roberta-base-features-question-6-7.h5py.data-00000-of-00002
/kaggle/input/robertaqa/test/roberta-base-features-question-0-7.h5py.index
/kaggle/input/robertaqa/test/roberta-base-features-question-4-7.h5py.data-00000-of-00002
/kaggle/input/robertaqa/test/roberta-base-features-question-2-7.h5py.data-00001-of-00002
/kaggle/input/robertaqa/test/roberta-base-features-question-0-7.h5py.data-00000-of-00002
/kaggle/input/robertaqa/test/roberta-base-features-question-4-7.h5py.index
/kaggle/input/robertaqa/test/roberta-

In [2]:
from urllib.parse import urlparse

import pandas as pd
import numpy as np
import re
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
import tensorflow_hub as hub
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm
import tensorflow as tf
import tensorflow.keras.backend as K
import gc
import os
from scipy.stats import spearmanr
from math import floor, ceil

np.set_printoptions(suppress=True)

import sys
tf.keras.backend.clear_session()
from transformers import *
print(tf.__version__)

2.1.0


In [3]:
MAX_SEQUENCE_LENGTH = 512

In [4]:
PATH = '../input/google-quest-challenge/'
# BERT_PATH = '../input/bert-base-from-tfhub/bert_en_uncased_L-12_H-768_A-12'
# tokenizer = tokenization.FullTokenizer(BERT_PATH+'/assets/vocab.txt', True)
BERT_PATH = '../input/robeata-base/'
#RBERT_PATH = '../input/robertabase/robert-base-tf_model'
tokenizer = RobertaTokenizer(BERT_PATH + "vocab.json", BERT_PATH + "roberta-base-merges.txt")


df_train = pd.read_csv(PATH+'train.csv')
df_test = pd.read_csv(PATH+'test.csv')

print('train shape =', df_train.shape)
print('test shape =', df_test.shape)

output_categories = list(df_train.columns[11:])
input_categories = list(df_train.columns[[1,2,5]])
#print('\noutput categories:\n\t', output_categories)
#print('\ninput categories:\n\t', input_categories)
output_categories_question =list(df_train.columns[11:32])
output_categories_answer = list(df_train.columns[32:])
input_categories_question = list(df_train.columns[[1,2]])
input_categories_answer = list(df_train.columns[[1,5]])

train shape = (6079, 41)
test shape = (476, 11)


In [5]:
def _get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens) > max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1] * len(tokens) + [0] * (max_seq_length - len(tokens))



def _get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [1] * (max_seq_length - len(token_ids))
    return input_ids


def _trim_input(title, question, max_sequence_length,
                t_max_len=100, head=128, tail=281, Q=True, q_max_len=239, a_max_len=239):
    t = tokenizer.tokenize(title)
    q = tokenizer.tokenize(question)
    # a = tokenizer.tokenize(answer)

    t_len = len(t)
    q_len = len(q)
    # a_len = len(a)

    if (t_len + q_len + 4) > max_sequence_length:
        if t_max_len > t_len:
            t_new_len = t_len
            q_head = head
            q_tail = 508 - q_head - t_new_len
        else:
            t_new_len = t_max_len
            if (t_new_len + q_len + 4) > max_sequence_length:
                q_head = head
                q_tail = 508 - q_head - t_new_len
            else:
                t = t[:t_new_len]

                return t, q
        t = t[:t_new_len]
        q = q[:q_head] + q[-q_tail:]
        return t, q
    else:
        return t, q


def _convert_to_bert_inputs(title, question, tokenizer, max_sequence_length):
    """Converts tokenized input to ids, masks and segments for BERT"""

    stoken = ["<s>"] + title + ["</s>"] + ["</s>"] + question + ["</s>"]

    input_ids = _get_ids(stoken, tokenizer, max_sequence_length)
    input_masks = _get_masks(stoken, max_sequence_length)
    #input_segments = _get_segments(stoken, max_sequence_length)

    return [input_ids, input_masks]


def compute_input_arays(df, columns, tokenizer, max_sequence_length, Q=True):
    input_ids, input_masks, input_segments = [], [], []
    for _, instance in tqdm(df[columns].iterrows()):
        if Q:
            t, q = instance.question_title, instance.question_body
            t, q = _trim_input(t, q, max_sequence_length)
            ids, masks= _convert_to_bert_inputs(t, q, tokenizer, max_sequence_length)
            input_ids.append(ids)
            input_masks.append(masks)

        else:
            t, a = instance.question_title, instance.answer
            t, a = _trim_input(t, a, max_sequence_length)
            ids, masks = _convert_to_bert_inputs(t, a, tokenizer, max_sequence_length)
            input_ids.append(ids)
            input_masks.append(masks)


    return [np.asarray(input_ids, dtype=np.int32),
            np.asarray(input_masks, dtype=np.int32)]


def compute_output_arrays(df, columns):
    return np.asarray(df[columns])

def compute_spearmanr(trues, preds):
    rhos = []
    for col_trues, col_pred in zip(trues.T, preds.T):
        rhos.append(
            spearmanr(col_trues, col_pred + np.random.normal(0, 1e-7, col_pred.shape[0])).correlation)
    return np.mean(rhos)


In [6]:
input_columns_answer = ['question_title', 'answer']
input_columns_question = ['question_title', 'question_body']

find = re.compile(r"^[^.]*")

df_train['netloc'] = df_train['url'].apply(lambda x: re.findall(find, urlparse(x).netloc)[0])
df_test['netloc'] = df_test['url'].apply(lambda x: re.findall(find, urlparse(x).netloc)[0])
features = ['netloc', 'category']
merged_answer = pd.concat([df_train[features], df_train[features]])
merged_question = pd.concat([df_train[features], df_train[features]])
ohe_answer = OneHotEncoder(handle_unknown='ignore')
ohe_question  = OneHotEncoder(handle_unknown='ignore')
ohe_answer.fit(merged_answer)
ohe_question.fit(merged_question)

OneHotEncoder(categorical_features=None, categories=None, drop=None,
              dtype=<class 'numpy.float64'>, handle_unknown='ignore',
              n_values=None, sparse=True)

In [7]:
for i in range(len(df_test)):
    if df_test.loc[i,'netloc'] not in ohe_answer.categories_[0]:
        df_test.loc[i,'netloc'] = 'stackoverflow'
    if df_test.loc[i,'category'] not in ohe_answer.categories_[1]:
        df_test.loc[i,'category'] = 'TECHNOLOGY'

In [8]:
features_test_answer = ohe_answer.transform(df_test[features]).toarray()
features_test_question = ohe_question.transform(df_test[features]).toarray()


module_url = "/kaggle/input/universalsentenceencoderlarge4/"
embed = hub.load(module_url)

embeddings_test = {}

for text in input_columns_answer:
    print(text)
    #train_text = df_train[text].str.replace('?', '.').str.replace('!', '.').tolist()
    test_text = df_test[text].str.replace('?', '.').str.replace('!', '.').tolist()

    #curr_train_emb = []
    curr_test_emb = []
    batch_size = 4
#     ind = 0
#     while ind * batch_size < len(train_text):
#         curr_train_emb.append(embed(train_text[ind * batch_size: (ind + 1) * batch_size])["outputs"].numpy())
#         ind += 1

    ind = 0
    while ind * batch_size < len(test_text):
        curr_test_emb.append(embed(test_text[ind * batch_size: (ind + 1) * batch_size])["outputs"].numpy())
        ind += 1

    #embeddings_train[text + '_embedding'] = np.vstack(curr_train_emb)
    embeddings_test[text + '_embedding'] = np.vstack(curr_test_emb)


del embed
K.clear_session()
gc.collect()

l2_dist = lambda x, y: np.power(x - y, 2).sum(axis=1)

cos_dist = lambda x, y: (x*y).sum(axis=1)

dist_features_test_answer = np.array([
    l2_dist(embeddings_test['question_title_embedding'], embeddings_test['answer_embedding']),
    cos_dist(embeddings_test['question_title_embedding'], embeddings_test['answer_embedding'])
]).T
feature_embeddings_answer = np.hstack([item for k, item in embeddings_test.items()] + [features_test_answer, dist_features_test_answer])


module_url = "/kaggle/input/universalsentenceencoderlarge4/"
embed = hub.load(module_url)

embeddings_test = {}
for text in input_columns_question:
    print(text)
    #train_text = df_train[text].str.replace('?', '.').str.replace('!', '.').tolist()
    test_text = df_test[text].str.replace('?', '.').str.replace('!', '.').tolist()

    #curr_train_emb = []
    curr_test_emb = []
    batch_size = 4
#     ind = 0
#     while ind * batch_size < len(train_text):
#         curr_train_emb.append(embed(train_text[ind * batch_size: (ind + 1) * batch_size])["outputs"].numpy())
#         ind += 1

    ind = 0
    while ind * batch_size < len(test_text):
        curr_test_emb.append(embed(test_text[ind * batch_size: (ind + 1) * batch_size])["outputs"].numpy())
        ind += 1

    #embeddings_train[text + '_embedding'] = np.vstack(curr_train_emb)
    embeddings_test[text + '_embedding'] = np.vstack(curr_test_emb)


del embed
K.clear_session()
gc.collect()

l2_dist = lambda x, y: np.power(x - y, 2).sum(axis=1)

cos_dist = lambda x, y: (x*y).sum(axis=1)



dist_features_test_question = np.array([
    l2_dist(embeddings_test['question_body_embedding'], embeddings_test['question_title_embedding']),
    cos_dist(embeddings_test['question_body_embedding'], embeddings_test['question_title_embedding'])
]).T




feature_embeddings_question = np.hstack([item for k, item in embeddings_test.items()] + [features_test_question, dist_features_test_question])

question_title
answer
question_title
question_body


In [9]:
feature_embeddings_answer_shape = feature_embeddings_answer.shape
feature_embeddings_question_shape = feature_embeddings_question.shape

In [10]:
def bert_model_question():
    input_word_ids = tf.keras.layers.Input(
        (MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = tf.keras.layers.Input(
        (MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    feature_embeddings = tf.keras.layers.Input(
        (feature_embeddings_question.shape[1],), dtype=tf.float32, name='feature_embeddings')
    
    bert_config = BertConfig.from_pretrained(BERT_PATH+"bert_config.json", output_hidden_states=True)
    bert_model = TFRobertaModel.from_pretrained(BERT_PATH+"roberta-base-tf_model.h5",config=bert_config)

    hidden_output = bert_model([input_word_ids, input_masks])[2]
    last_cat_sequence = tf.concat(
        (hidden_output[-1], hidden_output[-2], hidden_output[-3]),
        2,
    )
    x = tf.keras.layers.GlobalAveragePooling1D()(last_cat_sequence)
    x = tf.concat((x, feature_embeddings), 1)
    out = tf.keras.layers.Dense(21, activation="sigmoid", name="dense_output")(x)

    model = tf.keras.models.Model(
        inputs=[input_word_ids, input_masks, feature_embeddings], outputs=out)

    return model

In [11]:
def bert_model_answer():
    input_word_ids = tf.keras.layers.Input(
        (MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = tf.keras.layers.Input(
        (MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    feature_embeddings = tf.keras.layers.Input(
        (feature_embeddings_answer.shape[1],), dtype=tf.float32, name='feature_embeddings')

    bert_config = BertConfig.from_pretrained(BERT_PATH+"bert_config.json", output_hidden_states=True)
    bert_model = TFRobertaModel.from_pretrained(BERT_PATH+"roberta-base-tf_model.h5",config=bert_config)

    hidden_output = bert_model([input_word_ids, input_masks])[2]
    last_cat_sequence = tf.concat(
        (hidden_output[-1], hidden_output[-2], hidden_output[-3]),
        2,
    )
    x = tf.keras.layers.GlobalAveragePooling1D()(last_cat_sequence)
    x = tf.concat((x, feature_embeddings), 1)
    out = tf.keras.layers.Dense(9, activation="sigmoid", name="dense_output")(x)

    model = tf.keras.models.Model(
        inputs=[input_word_ids, input_masks, feature_embeddings], outputs=out)

    return model

In [12]:
test_inputs_question = compute_input_arays(df_test, input_categories_question, tokenizer, MAX_SEQUENCE_LENGTH)
test_inputs_question.append(feature_embeddings_question)

test_inputs_answer = compute_input_arays(df_test, input_categories_answer, tokenizer, MAX_SEQUENCE_LENGTH,Q=False)
test_inputs_answer.append(feature_embeddings_answer)

476it [00:01, 279.14it/s]
476it [00:01, 330.86it/s]


In [13]:
gc.collect()
os.listdir('../input')

['universalsentenceencoderlarge4',
 'google-quest-challenge',
 'robertaqa',
 'rfbanw8zhe',
 'robeata-base']

In [14]:
question_weights = []
for dirname, _, filenames in os.walk('../input/robertaqa/test/'):
    for filename in filenames:
        filename = os.path.join(dirname, filename)
        
        if ".h5py.index" in filename:
            filename = filename.split(".index")[0]
            question_weights.append(filename)

In [15]:
answer_weights = []
for dirname, _, filenames in os.walk('../input/rfbanw8zhe/test6/'):
    for filename in filenames:
        filename = os.path.join(dirname, filename)
        
        if ".h5py.index" in filename:
            filename = filename.split(".index")[0]
            answer_weights.append(filename)

In [16]:
answer_weights

['../input/rfbanw8zhe/test6/roberta-base-features-answer-6-6.h5py',
 '../input/rfbanw8zhe/test6/roberta-base-features-answer-7-6.h5py',
 '../input/rfbanw8zhe/test6/roberta-base-features-answer-2-6.h5py',
 '../input/rfbanw8zhe/test6/roberta-base-features-answer-4-6.h5py',
 '../input/rfbanw8zhe/test6/roberta-base-features-answer-5-6.h5py',
 '../input/rfbanw8zhe/test6/roberta-base-features-answer-0-6.h5py',
 '../input/rfbanw8zhe/test6/roberta-base-features-answer-1-6.h5py',
 '../input/rfbanw8zhe/test6/roberta-base-features-answer-3-6.h5py']

In [17]:
test_predictions_question = []
test_predictions_answer = []
for model_path in question_weights:
    K.clear_session()
    model = bert_model_question()
    print('loading.....{}'.format(model_path))
    model.load_weights(model_path)
    print('predicting......')
    test_predictions_question.append(model.predict(test_inputs_question, batch_size=32,))

loading......./input/robertaqa/test/roberta-base-features-question-0-7.h5py
predicting......
loading......./input/robertaqa/test/roberta-base-features-question-4-7.h5py
predicting......
loading......./input/robertaqa/test/roberta-base-features-question-3-7.h5py
predicting......
loading......./input/robertaqa/test/roberta-base-features-question-2-7.h5py
predicting......
loading......./input/robertaqa/test/roberta-base-features-question-6-7.h5py
predicting......
loading......./input/robertaqa/test/roberta-base-features-question-7-7.h5py
predicting......
loading......./input/robertaqa/test/roberta-base-features-question-5-7.h5py
predicting......
loading......./input/robertaqa/test/roberta-base-features-question-1-7.h5py
predicting......


In [18]:
for model_path in answer_weights:
    K.clear_session()
    model = bert_model_answer()
    print('loading.....{}'.format(model_path))
    model.load_weights(model_path)
    print('predicting......')
    test_predictions_answer.append(model.predict(test_inputs_answer, batch_size=32))

loading......./input/rfbanw8zhe/test6/roberta-base-features-answer-6-6.h5py
predicting......
loading......./input/rfbanw8zhe/test6/roberta-base-features-answer-7-6.h5py
predicting......
loading......./input/rfbanw8zhe/test6/roberta-base-features-answer-2-6.h5py
predicting......
loading......./input/rfbanw8zhe/test6/roberta-base-features-answer-4-6.h5py
predicting......
loading......./input/rfbanw8zhe/test6/roberta-base-features-answer-5-6.h5py
predicting......
loading......./input/rfbanw8zhe/test6/roberta-base-features-answer-0-6.h5py
predicting......
loading......./input/rfbanw8zhe/test6/roberta-base-features-answer-1-6.h5py
predicting......
loading......./input/rfbanw8zhe/test6/roberta-base-features-answer-3-6.h5py
predicting......


In [19]:
final_predictions_question = np.mean(test_predictions_question, axis=0)
final_predictions_answer = np.mean(test_predictions_answer, axis=0)

In [20]:
gc.collect()
df_sub = pd.read_csv(PATH+'sample_submission.csv')

In [21]:
df_train[output_categories]

,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,question_not_really_a_question,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,1.000000,0.333333,0.000000,0.000000,0.000000,0.0,1.000000,1.000000,0.000000,0.0,...,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.000000,0.000000,0.000000,1.000000
1,1.000000,1.000000,0.000000,0.500000,1.000000,1.0,0.444444,0.444444,0.666667,0.0,...,0.888889,0.888889,0.555556,0.888889,0.888889,0.666667,0.000000,0.000000,0.666667,0.888889
2,0.888889,0.666667,0.000000,1.000000,1.000000,1.0,0.666667,0.444444,0.333333,0.0,...,0.777778,0.777778,0.555556,1.000000,1.000000,0.666667,0.000000,0.333333,1.000000,0.888889
3,0.888889,0.666667,0.666667,1.000000,1.000000,1.0,0.444444,0.444444,0.000000,0.0,...,0.888889,0.833333,0.333333,0.833333,1.000000,0.800000,0.000000,0.000000,1.000000,1.000000
4,1.000000,0.666667,0.000000,1.000000,1.000000,1.0,0.666667,0.666667,0.000000,0.0,...,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.000000,0.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6074,1.000000,0.777778,0.000000,1.000000,1.000000,1.0,0.888889,0.888889,0.333333,0.0,...,0.777778,1.000000,0.555556,1.000000,1.000000,0.866667,0.000000,0.000000,0.000000,0.888889
6075,1.000000,0.777778,0.000000,1.000000,0.666667,0.5,0.777778,0.777778,0.000000,0.0,...,0.777778,0.888889,0.555556,1.000000,1.000000,0.733333,0.666667,0.333333,0.000000,0.888889
6076,0.888889,0.555556,0.000000,0.666667,0.333333,1.0,0.444444,0.333333,0.000000,0.0,...,0.444444,0.888889,0.555556,0.888889,0.888889,0.800000,1.000000,0.000000,0.333333,0.555556
6077,1.000000,0.444444,0.333333,0.000000,0.000000,0.0,0.777778,0.555556,1.000000,0.0,...,1.000000,0.888889,0.555556,1.000000,1.000000,0.533333,0.000000,0.333333,0.666667,1.000000


In [22]:
df_sub.iloc[:, 1:20] = np.around(final_predictions_question[:,:19] *99) / 99
df_sub.iloc[:, 21] = np.around(final_predictions_question[:,20] *99) / 99
df_sub.iloc[:, 20] = np.around(final_predictions_question[:,19] *9000) / 9000
df_sub.iloc[:, 22:] = np.around(final_predictions_answer * 99) / 99
pd.set_option('display.max_columns', None)
df_sub.head(40)

# df_sub.iloc[:, 1:20] = final_predictions_question[:,:19]
# df_sub.iloc[:, 21] = final_predictions_question[:,20]
# df_sub.iloc[:, 20] = final_predictions_question[:,19] 
# df_sub.iloc[:, 22:] = final_predictions_answer
# pd.set_option('display.max_columns', None)
# df_sub.head(40)

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,question_not_really_a_question,question_opinion_seeking,question_type_choice,question_type_compare,question_type_consequence,question_type_definition,question_type_entity,question_type_instructions,question_type_procedure,question_type_reason_explanation,question_type_spelling,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.959596,0.707071,0.181818,0.575758,0.636364,0.595960,0.666667,0.606061,0.616162,0.000000,0.717172,0.696970,0.010101,0.141414,0.000000,0.000000,0.080808,0.070707,0.727273,0.000333,0.949495,0.939394,0.636364,0.969697,0.969697,0.868687,0.000000,0.010101,0.707071,0.959596
1,46,0.868687,0.545455,0.000000,0.828283,0.757576,0.959596,0.535354,0.494949,0.141414,0.000000,0.434343,0.383838,0.000000,0.000000,0.000000,0.000000,0.949495,0.191919,0.111111,0.000222,0.555556,0.919192,0.626263,0.959596,0.979798,0.858586,0.909091,0.191919,0.070707,0.898990
2,70,0.939394,0.707071,0.010101,0.777778,0.959596,0.979798,0.575758,0.454545,0.121212,0.000000,0.060606,0.737374,0.020202,0.050505,0.000000,0.000000,0.030303,0.060606,0.575758,0.001111,0.898990,0.959596,0.595960,0.979798,0.979798,0.848485,0.050505,0.040404,0.888889,0.939394
3,132,0.939394,0.454545,0.000000,0.797980,0.707071,0.929293,0.545455,0.434343,0.070707,0.000000,0.555556,0.020202,0.000000,0.000000,0.000000,0.000000,0.929293,0.181818,0.393939,0.000333,0.717172,0.929293,0.707071,0.959596,0.969697,0.878788,0.717172,0.131313,0.626263,0.898990
4,200,0.949495,0.444444,0.030303,0.919192,0.757576,0.949495,0.616162,0.616162,0.202020,0.010101,0.424242,0.787879,0.010101,0.020202,0.000000,0.010101,0.090909,0.101010,0.505050,0.000556,0.747475,0.929293,0.666667,0.979798,0.969697,0.848485,0.343434,0.242424,0.444444,0.898990
5,245,0.919192,0.858586,0.010101,0.848485,0.969697,0.979798,0.626263,0.454545,0.161616,0.000000,0.070707,0.010101,0.000000,0.010101,0.010101,0.010101,0.020202,0.171717,0.949495,0.000889,0.929293,0.989899,0.686869,0.989899,0.989899,0.949495,0.050505,0.111111,0.727273,0.949495
6,257,0.898990,0.444444,0.000000,0.898990,0.585859,0.949495,0.545455,0.383838,0.010101,0.000000,0.606061,0.010101,0.000000,0.000000,0.000000,0.000000,0.949495,0.131313,0.030303,0.000222,0.686869,0.959596,0.676768,0.969697,0.989899,0.919192,0.878788,0.181818,0.565657,0.909091
7,267,0.959596,0.696970,0.333333,0.848485,0.717172,0.737374,0.656566,0.747475,0.161616,0.000000,0.545455,0.181818,0.010101,0.000000,0.020202,0.000000,0.000000,0.000000,0.969697,0.002444,0.898990,0.909091,0.676768,0.969697,0.969697,0.858586,0.010101,0.000000,0.969697,0.929293
8,284,0.878788,0.434343,0.000000,0.686869,0.757576,0.868687,0.535354,0.474747,0.555556,0.000000,0.616162,0.616162,0.010101,0.000000,0.000000,0.000000,0.929293,0.232323,0.222222,0.000556,0.797980,0.979798,0.686869,0.989899,0.989899,0.949495,0.878788,0.111111,0.383838,0.939394
9,292,0.979798,0.626263,0.010101,0.868687,0.797980,0.868687,0.727273,0.636364,0.131313,0.000000,0.363636,0.010101,0.000000,0.000000,0.000000,0.070707,0.383838,0.070707,0.848485,0.000556,0.898990,0.949495,0.646465,0.989899,0.989899,0.888889,0.747475,0.050505,0.939394,0.959596


In [23]:
#Improving Discretization by Post- Processing Procedure
def minmaxscaler(num,max_num,min_num):
    
    new_num = (num-min_num)/(max_num-min_num)
    return new_num

train_unique_dict = {}
for column in output_categories:
    test_unique = np.sort(df_train[column].unique().tolist())
    new_unique = []
    min_num = np.min(test_unique)
    max_num = np.max(test_unique) #find min and max for each output column

    for num in test_unique:
        new_unique.append(minmaxscaler(num,max_num,min_num))
    train_unique_dict[column] = new_unique 
# train_unique_dict

In [24]:
def distance(num,num_list):
    
    min_dis_num = 1e9
    nearest_num_idx = 0
    
    for idx,n in enumerate(num_list):
        dis_num = np.abs(num-n)
        if dis_num <= min_dis_num:
            min_dis_num = dis_num
            nearest_num_idx = idx
    return nearest_num_idx

for i in output_categories:
    new_data = []
    train_unique = np.sort(df_train[i].unique().tolist())
    test_max = np.max(df_sub[i])
    test_min = np.min(df_sub[i])
    dif_value = test_max - test_min
    test_num_list = []
    
    for j in train_unique_dict[i]:
        test_num_list.append(test_min+dif_value*j)
#     print(i,test_max,test_min,test_num_list,'---->',train_unique_dict[i])
    test_num_list = np.sort(test_num_list)
    
    for num in df_sub[i]:
        new_num_idx = distance(num,test_num_list)
        new_data.append(train_unique[new_num_idx])
    df_sub[i] = new_data

In [25]:
df_sub

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,question_not_really_a_question,question_opinion_seeking,question_type_choice,question_type_compare,question_type_consequence,question_type_definition,question_type_entity,question_type_instructions,question_type_procedure,question_type_reason_explanation,question_type_spelling,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.888889,0.777778,0.333333,0.500000,0.500000,0.500000,0.777778,0.777778,0.666667,0.000000,0.666667,0.666667,0.0,0.333333,0.0,0.0,0.000000,0.000000,0.666667,0.0,1.000000,0.888889,0.666667,0.888889,0.888889,0.800000,0.000000,0.000000,0.666667,1.000000
1,46,0.666667,0.666667,0.000000,0.666667,0.666667,1.000000,0.500000,0.555556,0.000000,0.000000,0.500000,0.333333,0.0,0.000000,0.0,0.0,1.000000,0.333333,0.000000,0.0,0.333333,0.833333,0.666667,0.833333,0.888889,0.800000,1.000000,0.500000,0.000000,0.833333
2,70,0.888889,0.777778,0.000000,0.666667,1.000000,1.000000,0.555556,0.555556,0.000000,0.000000,0.000000,0.666667,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.500000,0.0,0.888889,0.888889,0.555556,1.000000,0.888889,0.733333,0.000000,0.000000,1.000000,0.888889
3,132,0.888889,0.555556,0.000000,0.666667,0.666667,1.000000,0.500000,0.500000,0.000000,0.000000,0.666667,0.000000,0.0,0.000000,0.0,0.0,1.000000,0.333333,0.333333,0.0,0.666667,0.888889,0.833333,0.833333,0.888889,0.800000,0.666667,0.333333,0.666667,0.833333
4,200,0.888889,0.500000,0.000000,1.000000,0.666667,1.000000,0.666667,0.777778,0.333333,0.333333,0.500000,0.666667,0.0,0.000000,0.0,0.0,0.000000,0.333333,0.500000,0.0,0.666667,0.888889,0.666667,1.000000,0.888889,0.733333,0.333333,0.500000,0.500000,0.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,9569,0.666667,0.666667,0.000000,0.666667,0.500000,1.000000,0.500000,0.444444,0.000000,0.000000,0.500000,0.000000,0.0,0.000000,0.0,0.0,1.000000,0.333333,0.000000,0.0,0.666667,0.888889,0.666667,0.833333,0.888889,0.866667,1.000000,0.333333,0.000000,0.833333
472,9590,0.777778,0.555556,0.000000,0.500000,0.666667,0.666667,0.555556,0.555556,0.000000,0.000000,0.500000,0.000000,0.0,0.000000,0.0,0.0,0.666667,1.000000,0.000000,0.0,0.444444,0.888889,0.666667,0.833333,0.888889,0.866667,1.000000,0.333333,0.000000,0.833333
473,9597,0.666667,0.333333,0.000000,0.666667,0.500000,0.666667,0.444444,0.500000,0.500000,0.000000,0.666667,0.666667,0.0,0.000000,0.0,0.0,0.500000,0.500000,0.333333,0.0,0.666667,0.888889,0.555556,1.000000,0.888889,0.800000,0.666667,0.333333,0.500000,0.888889
474,9623,0.888889,0.833333,0.000000,1.000000,1.000000,1.000000,0.666667,0.555556,0.000000,0.000000,0.000000,1.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.888889,1.000000,0.833333,1.000000,1.000000,1.000000,0.000000,0.333333,1.000000,1.000000


In [26]:
for i in output_categories:
    print(len(df_sub[i].unique()),len(df_train[i].unique()),i)

9 9 question_asker_intent_understanding
9 9 question_body_critical
5 5 question_conversational
5 5 question_expect_short_answer
5 5 question_fact_seeking
5 5 question_has_commonly_accepted_answer
9 9 question_interestingness_others
9 9 question_interestingness_self
5 5 question_multi_intent
4 5 question_not_really_a_question
5 5 question_opinion_seeking
5 5 question_type_choice
5 5 question_type_compare
5 5 question_type_consequence
5 5 question_type_definition
5 5 question_type_entity
5 5 question_type_instructions
5 5 question_type_procedure
5 5 question_type_reason_explanation
3 3 question_type_spelling
9 9 question_well_written
9 9 answer_helpful
9 9 answer_level_of_information
9 9 answer_plausible
9 9 answer_relevance
16 17 answer_satisfaction
5 5 answer_type_instructions
5 5 answer_type_procedure
5 5 answer_type_reason_explanation
8 9 answer_well_written


In [27]:
df_sub.to_csv('submission.csv', index=False)